# Improved Implementation: RoBERTa+BiLSTM

Using best parameters on the full training dataset with metadata, the performance metric scores on the test dataset are as follows: 
1. ACC: 0.8044
2. F1: 0.8017
3. MCC: 0.6198

## Import Libraries

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
# from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from tqdm import tqdm
import itertools
import pandas as pd

## Set Device
Set device to CUDA if available, else CPU

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
if device == 'cuda': 
    torch.cuda.empty_cache()

## Loading Data (Test, Train, Validate)

In [4]:
# User Metadata
train_metadata = torch.load("../Data/Processed_Data/train_metadata_tensor.pth").to(device)
test_metadata = torch.load("../Data/Processed_Data/test_metadata_tensor.pth").to(device)
# validate_metadata = torch.load("../Data/Processed_Data/validate_metadata_tensor.pth").to(device)

# Tweets Data
train_tweets = torch.load("../Data/Processed_Data/train_tweet_roberta_emb_tensor.pth")[:,:200].to(device)
test_tweets = torch.load("../Data/Processed_Data/test_tweet_roberta_emb_tensor.pth")[:,:200].to(device)
# validate_tweets = torch.load("../Data/Processed_Data/validate_tweet_roberta_emb_tensor.pth").to(device)

# Labels
train_labels = torch.load("../Data/Processed_Data/train_label_tensor.pth").to(device)
test_labels = torch.load("../Data/Processed_Data/test_label_tensor.pth").to(device)
# validate_labels = torch.load("../Data/Processed_Data/validate_label_tensor.pth").to(device)

In [5]:
print(train_tweets.shape)
print(test_tweets.shape)
# print(validate_tweets.shape)

print(train_metadata.shape)
print(test_metadata.shape)
# print(validate_metadata.shape)

torch.Size([1398465, 200])
torch.Size([199863, 200])
torch.Size([1398465, 5])
torch.Size([199863, 5])


Set seed for Reproducability

In [6]:
seed = 42
torch.manual_seed(seed)

Stratified sampling to expedite model training

In [ ]:
"""
# Move labels to CPU for processing
train_labels_cpu = train_labels.cpu().numpy()
# train_labels_cpu = test_labels.cpu().numpy()

# Perform stratified sampling to maintain label distribution
_, sampled_indices = train_test_split(
    range(train_tweets.shape[0]), 
    # range(test_tweets.shape[0]), 
    test_size=0.3, 
    stratify=train_labels_cpu, 
    random_state=42
)

# Convert to tensor and move to device
sampled_indices = torch.tensor(sampled_indices, device=device)

train_tweets = train_tweets[sampled_indices].contiguous()
# train_metadata = train_metadata[sampled_indices].contiguous()
train_labels = train_labels[sampled_indices].contiguous()

# test_tweets = test_tweets[sampled_indices].contiguous()
# train_metadata = train_metadata[sampled_indices].contiguous()
# test_labels = test_labels[sampled_indices].contiguous()

print(train_tweets.shape)
"""


## BiLSTM Model

In [7]:
class LSTMModel(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout_rate = 0.0):
        super(LSTMModel, self).__init__()
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=False, bidirectional = True, dropout = dropout_rate)
        
        # Define a fully connected output layer
        self.fc = nn.Linear(hidden_size * 2, output_size)
        
        
    def forward(self, x):
        # Pass through BiLSTM
        lstm_out, (hn, cn) = self.lstm(x)
        
        output = self.fc(lstm_out[:, -1, :])
        return output

In [8]:
lstm_input_size = 200 #768 # Number of features in input data (768 for Roberta Embeddings)
lstm_hidden_size = 50
lstm_output_size = 32
lstm_number_layers = 1 # Number of LSTM Layers
lstm_dropout = 0

# Creating LSTM Model
lstm_model = LSTMModel(input_size = lstm_input_size, hidden_size = lstm_hidden_size, output_size = lstm_output_size, 
                      num_layers = lstm_number_layers, dropout_rate=lstm_dropout).to(device)

## Prepare Data

Getting lstm_output for datasets

In [9]:
train_tweets = train_tweets.unsqueeze(1).to(torch.float32)
test_tweets = test_tweets.unsqueeze(1).to(torch.float32)
                
train_lstm_output = lstm_model(train_tweets)
train_lstm_output = train_lstm_output.detach()

test_lstm_output = lstm_model(test_tweets)
test_lstm_output = test_lstm_output.detach()

In [10]:
print(train_lstm_output.shape)
print(test_lstm_output.shape)

torch.Size([1398465, 32])
torch.Size([199863, 32])


Concatenate text embedding with metadata embedding

In [11]:
x_train = torch.hstack((train_lstm_output, train_metadata)).to(torch.float32)
x_test = torch.hstack((test_lstm_output, test_metadata)).to(torch.float32)

In [12]:
y_train = train_labels.view(-1, 1).to(torch.float32)
y_test = test_labels.view(-1, 1).to(torch.float32)

In [13]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

torch.Size([1398465, 37]) torch.Size([1398465, 1])
torch.Size([199863, 37]) torch.Size([199863, 1])


In [14]:
x_train.requires_grad_(False)
y_train.requires_grad_(False)

tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]], device='cuda:0')

## Final Layer

In [15]:
class NeuralNetwork(nn.Module):
    
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size=1, dropout=0.2):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size_1)  # First fully connected layer
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)  # Second fully connected layer
        self.out = nn.Linear(hidden_size_2, output_size)
        self.dropout = nn.Dropout(p = dropout)
        
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Apply ReLU activation after the first layer
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))  # Output layer
        
        x = self.out(x)
        return x

In [16]:
# Network Parameters
input_size = 37 # Number of input features (tweet embeddings + metadata)
hidden_size_1 = 128  # Number of hidden units in the first fully connected layer
hidden_size_2 = 64  # Number of hidden units in the first fully connected layer
output_size = 1  # Output size (binary classification: 1 output)

## Create Dataloaders

In [17]:
batch_size = 64 # Batch size for training

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

## Model Training

Initialize model, loss function, and optimizer

In [18]:
torch.manual_seed(seed)
model = NeuralNetwork(input_size, hidden_size_1, hidden_size_2, output_size).to(device)
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy with logits
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Train model

In [19]:
def train_model(model, train_loader, optimizer, criterion, epochs=10):
    """
    :param model: bot classifier
    :param train_loader: training dataset
    :param optimizer: Adam as specified on paper
    :param criterion: BCE as specified on paper
    :param epochs: adjust for hyperparameter tuning
    :return: None
    """
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True, dynamic_ncols=True, position=0) 
        
        for i, batch in enumerate(progress_bar):
            embeddings, labels = batch[0].float().to(device), batch[1].float().to(device)
            optimizer.zero_grad()
            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            if i % 10 == 0:
                progress_bar.set_postfix(loss=f"{loss.item():.4f}")

In [25]:
train_model(model, train_loader, optimizer, criterion, epochs=10)

Epoch 10/10: 100%|██████████| 21852/21852 [00:46<00:00, 473.38it/s, loss=0.4773]


## Model Evaluation

Evaluate model

In [20]:
def evaluate_model(model, test_loader):
    """
    :param model: bot classifier
    :param test_loader: testing dataset
    :return: None
    """
    model.eval()
    preds, true_labels = [], []
    
    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Evaluating", unit="batch", dynamic_ncols=True, position=0)
        
        for batch in progress_bar:
            embeddings, labels = batch[0].float().to(device), batch[1].float().to(device)
            outputs = model(embeddings)
            probs = torch.sigmoid(outputs)
            preds.extend((probs > 0.5).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    acc = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')  # Adjust 'weighted' if needed
    mcc = matthews_corrcoef(true_labels, preds)
    
    return acc, f1, mcc

In [27]:
evaluate_model(model, test_loader)

Evaluating: 100%|██████████| 3123/3123 [00:02<00:00, 1172.49batch/s]


(0.8044960798146731, 0.8017276384516153, 0.6198985535659354)